In [1]:
import numpy as np

np.random.seed(1)

# Y = XW + b (simple forward pass)

In [2]:
def matmul_float(X, W, b):
    return X @ W + b

def parameters(Qmin, Qmax, Xmin, Xmax, Wmin, Wmax, bmin, bmax):
    q_range = Qmax - Qmin
    sx = (Xmax - Xmin) / q_range
    sw = (Wmax - Wmin) / q_range
    sb = (bmax - bmin) / q_range
    zx = np.round((Xmax * Qmin - Xmin * Qmax) / (Xmax - Xmin))
    zw = np.round((Wmax * Qmin - Wmin * Qmax) / (Wmax - Wmin))
    zb = np.round((bmax * Qmin - bmin * Qmax) / (bmax - bmin))
    return sx, sw, sb, zx, zw, zb

def quantize(N, s, z, Qmin, Qmax):
    return np.clip(np.round(N / s + z), Qmin, Qmax)

def dequantize(Nq, s, z):
    return np.float32(s * (Nq - z))

def matmul_int8(Xq, Wq, bq, sx, sw, sb, zx, zw, zb):
    product = np.int8(sx * sw * (Xq @ Wq))
    # Note: summation along the P axis
    # [MxP]
    inputs_scale = np.int8(sx * sw * zw * np.sum(Xq, axis=1, keepdims=True))
    # [PxN]
    weights_scale = np.int8(sx * sw * zx * np.sum(Wq, axis=0, keepdims=True))
    offsets_scale = np.int8(sx * sw * zx * zw)
    biases_scale = np.int8(sb * (bq - zb))
    print(f'product: {product.shape}\n')
    print(f'inputs_scale: {inputs_scale.shape}\n')
    print(f'weights_scale: {weights_scale.shape}\n')
    print(f'offsets_scale: {offsets_scale}\n')
    print(f'biases_scale: {biases_scale.shape}\n')
    return (product - inputs_scale - weights_scale + offsets_scale + biases_scale).astype(np.int8)

In [3]:
M = 2;N = 4;P = 4;

Xmin = 10; Xmax = 50
Wmin = 0; Wmax = 1
bmin = 10; bmax = 30

# [MxP] - 2x3
X = np.random.uniform(10, 50, (M, P)).astype(np.float32)
# [PxN] - 3x4
W = np.random.uniform(0, 1, (P, N)).astype(np.float32)
# [1xN] - 1x4
b = np.random.uniform(10, 30, (1, N)).astype(np.float32)

print(f'X: {X}\n')
print(f'W: {W}\n')
print(f'b: {b}\n')

print(f'X: {X.shape}\n')
print(f'W: {W.shape}\n')
print(f'b: {b.shape}\n')

X: [[26.68088  38.81298  10.004575 22.093304]
 [15.870235 13.693543 17.450409 23.82243 ]]

W: [[0.39676747 0.53881675 0.41919452 0.6852195 ]
 [0.20445225 0.87811744 0.02738759 0.6704675 ]
 [0.4173048  0.55868983 0.14038694 0.19810149]
 [0.8007446  0.9682616  0.31342417 0.6923226 ]]

b: [[27.527782  27.892134  11.700884  10.7810955]]

X: (2, 4)

W: (4, 4)

b: (1, 4)



In [4]:
Y_float = matmul_float(X, W, b)
print(f'Y_float: {Y_float}\n')
print(f'Y_float: {Y_float.shape}\n')

Y_float: [[ 67.915344 103.332146  32.277443  72.36381 ]
 [ 62.98207   81.28353   28.644968  50.78653 ]]

Y_float: (2, 4)



In [5]:
bit = 8
Qmin = -2**(bit - 1) + 1
Qmax = 2**(bit - 1) - 1
print(f'quantization: [{Qmin}, {Qmax}]\n')
sx, sw, sb, zx, zw, zb = parameters(Qmin, Qmax, Xmin, Xmax, Wmin, Wmax, bmin, bmax)

print(f'parameters: {sx}, {sw}, {sb}, {zx}, {zw}, {zb}\n')

Xq = quantize(X, sx, zx, Qmin, Qmax)
Wq = quantize(W, sw, zw, Qmin, Qmax)
bq = quantize(b, sb, zb, Qmin, Qmax)

print(f'Xq: {Xq}\n')
print(f'Wq: {Wq}\n')
print(f'bq: {bq}\n')

quantization: [-127, 127]

parameters: 0.15748031496062992, 0.003937007874015748, 0.07874015748031496, -190.0, -127.0, -254.0

Xq: [[ -21.   56. -126.  -50.]
 [ -89. -103.  -79.  -39.]]

Wq: [[ -26.   10.  -21.   47.]
 [ -75.   96. -120.   43.]
 [ -21.   15.  -91.  -77.]
 [  76.  119.  -47.   49.]]

bq: [[  96.  100. -105. -117.]]



In [6]:
Y_int8 = matmul_int8(Xq, Wq, bq, sx, sw, sb, zx, zw, zb)
print(f'Y_int8: {Y_int8}\n')

product: (2, 4)

inputs_scale: (2, 1)

weights_scale: (1, 4)

offsets_scale: 14

biases_scale: (1, 4)

Y_int8: [[ 23  57 -14  25]
 [ 17  35 -17   5]]



In [7]:
Ymin = 0;Ymax = 110
q_range = Qmax - Qmin
s = (Ymax - Ymin) / q_range
z = np.round((Ymax * Qmin - Ymin * Qmax) / (Ymax - Ymin))
Y_dequantized_float = dequantize(Y_int8, s, z)
print(f'Y_dequantized_float: {Y_dequantized_float}')

Y_dequantized_float: [[64.96063  79.685036 48.937008 65.826775]
 [62.362206 70.15748  47.637794 57.165356]]
